In [1]:
import re
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
# import seaborn as sns

## First Step: Select Airbnb Homes for Research

In [6]:
fields = ['id', 'price', 'last_scraped','number_of_reviews', 'review_scores_rating', 
          'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 
          'review_scores_communication', 'review_scores_location', 'review_scores_value', 
          'reviews_per_month']
homes_df = pd.read_csv("./Data/listings.csv", encoding = "utf-8", usecols = fields)

homes_df.columns = ['home_id', 'last_scraped', 'price', 'number_of_reviews',
                    'scores_overall', 'scores_accuracy', 'scores_cleanliness', 
                    'scores_checkin', 'scores_communication', 'scores_location', 
                    'scores_value', 'reviews_per_month']
homes_df = homes_df.dropna() # removed 660 homes which are lack of some score information
homes_df.head(3)

,home_id,last_scraped,price,number_of_reviews,scores_overall,scores_accuracy,scores_cleanliness,scores_checkin,scores_communication,scores_location,scores_value,reviews_per_month
0,241032,2016-01-04,$85.00,207,95.0,10.0,10.0,10.0,10.0,9.0,10.0,4.07
1,953595,2016-01-04,$150.00,43,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1.48
2,3308979,2016-01-04,$975.00,20,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.15


In [7]:
# change price from string to integer such as "$85.00" to "85"
def extract_price(s):
    return int(''.join(re.findall(r"\d+", s.split('.')[0])))

homes_df.price = [extract_price(p) for p in homes_df.price]
homes_df.head(3)

,home_id,last_scraped,price,number_of_reviews,scores_overall,scores_accuracy,scores_cleanliness,scores_checkin,scores_communication,scores_location,scores_value,reviews_per_month
0,241032,2016-01-04,85,207,95.0,10.0,10.0,10.0,10.0,9.0,10.0,4.07
1,953595,2016-01-04,150,43,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1.48
2,3308979,2016-01-04,975,20,97.0,10.0,10.0,10.0,10.0,10.0,10.0,1.15


In [9]:
print(max(homes_df.last_scraped))
print(min(homes_df.last_scraped))

2016-01-04
2016-01-04


In [10]:
print("There are " + str(len(homes_df)) + " airbnb homes.")
print("The price ranging from $" + str(min(homes_df.price)) + " to $" 
      + str(max(homes_df.price)) + ".")

There are 3158 airbnb homes.
The price ranging from $25 to $1000.


In [12]:
# len(homes_sample_df)
len(homes_df)

3158

In [13]:
homes_df.to_csv('./Data/selected_data_for_research.csv', sep='\t', encoding='utf-8')

**********************************************************************************************
## Next Step: Combine selected homes with its reviews

In [14]:
# Load Necessary Data: reviews
reviews_df = pd.read_csv("./Data/reviews.csv", encoding="utf-8")
reviews_df.columns = ['home_id', 'review_id', 'date', 'reviewer_id', 'reviewer_name', 'comments']
reviews_df.dropna()
reviews_df.head(2)
len(reviews_df)

84849

In [15]:
print(min(reviews_df.date), max(reviews_df.date))
# reviews_df.groupby('comments').count()
reviews_df = reviews_df[reviews_df.comments != 'The host canceled this reservation the day before arrival. This is an automated posting.']
len(reviews_df)

2009-06-07 2016-01-03


84792

In [16]:
from langdetect import detect
def lang_detection(series):
    isEn = []
    for text in series:
        try:
            language = detect(text)
        except:
            language = "error"
            print("This row throws and error:", text)
        
        if language == 'en':
            isEn.append(True)
        else:
            isEn.append(False)
            
    return isEn
#     return [True if detect(text) == 'en' else False for text in series]

In [17]:
lang_isen = lang_detection(reviews_df.comments)

This row throws and error: .
This row throws and error: ...
This row throws and error: :)
This row throws and error: -
This row throws and error: nan
This row throws and error: :)
This row throws and error: nan
This row throws and error: nan
This row throws and error: nan
This row throws and error: nan
This row throws and error: *
This row throws and error: nan
This row throws and error: nan
This row throws and error: nan
This row throws and error: .
This row throws and error: nan
This row throws and error: nan
This row throws and error: nan
This row throws and error: nan
This row throws and error: nan
This row throws and error: .
This row throws and error: nan
This row throws and error: nan
This row throws and error: nan
This row throws and error: ..
This row throws and error: .
This row throws and error: .
This row throws and error: nan
This row throws and error: nan
This row throws and error: .


In [18]:
reviews_df = reviews_df[lang_isen]

In [19]:
reviews_df.head()

,home_id,review_id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [20]:
sample1_df = pd.read_csv("./Data/selected_data_for_research.csv", sep='\t', encoding="utf-8")
sample1_df = sample1_df.drop("Unnamed: 0", axis=1)
sample1_df.head(2)

,home_id,last_scraped,price,number_of_reviews,scores_overall,scores_accuracy,scores_cleanliness,scores_checkin,scores_communication,scores_location,scores_value,reviews_per_month
0,241032,2016-01-04,85,207,95.0,10.0,10.0,10.0,10.0,9.0,10.0,4.07
1,953595,2016-01-04,150,43,96.0,10.0,10.0,10.0,10.0,10.0,10.0,1.48


In [21]:
# Merge the reviews and homes in the sample data.
df1 = sample1_df[['home_id', 'scores_cleanliness', 'scores_location']]
df2 = reviews_df[['home_id', 'review_id', 'comments']]
sample1_rh_df = pd.merge(df1, df2, on="home_id")
sample1_rh_df.head(3)
# sample1_rh_df.stack()[0].comments

,home_id,scores_cleanliness,scores_location,review_id,comments
0,241032,10.0,9.0,682061,Excellent all the way around. \r\n\r\nMaija wa...
1,241032,10.0,9.0,691712,Maija's apartment was a wonderful place to sta...
2,241032,10.0,9.0,702999,one of the most pleasant stays i've had in my ...


In [22]:
# set(sample1_rh_df.home_id)
# set(homes_df)
set(homes_df.home_id) - set(sample1_rh_df.home_id)
# print(len(set(sample1_rh_df.home_id)), len(homes_df))

{7873208, 8097708, 9558353}

In [25]:
homes_df[homes_df.home_id == 7873208]
homes_df[homes_df.home_id == 9558353]

,home_id,last_scraped,price,number_of_reviews,scores_overall,scores_accuracy,scores_cleanliness,scores_checkin,scores_communication,scores_location,scores_value,reviews_per_month
760,9558353,2016-01-04,70,1,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0.77


**********************************************************************************************
## Last Step: Get homes and reviews data for sentence analysis
### Find data for `cleanliness` and `location` aspects 
**********************************************************************************************
#### Brief Summary & Necessary Functions

In [23]:
# sample2_rh_df is a copy of sample1_rh_df to 
# in case unexpected modification for original data.
sample2_rh_df = sample1_rh_df
print('*' * 40 + '\nThere are:\n' + '-' * 40)
print(str(len(sample2_rh_df.groupby('home_id'))) + " Airbnb homes in total.\n" + '-' * 40)
print(str(len(sample2_rh_df)) + " reviews in total.\n" + '-' * 40)

****************************************
There are:
----------------------------------------
3155 Airbnb homes in total.
----------------------------------------
83682 reviews in total.
----------------------------------------


In [31]:
# group by aspect scores
def overview_df(aspect):
    df1 = pd.DataFrame(sample2_rh_df.groupby([aspect])['home_id'].nunique())
    df2 = pd.DataFrame(sample2_rh_df.groupby([aspect])['review_id'].nunique())
    overview_df = pd.merge(df1, df2, on = aspect)
    overview_df.columns = ['number of homes', 'number of reveiws']
    return overview_df
# what makes people comments

# Input: the expecting aspect and score
# Output: the DataFrame with selected homes and their reviews information
def home_sample_review(score, aspect, overview_df):
    num_home = overview_df.loc[score].values[0]
#     num_review = overview_df.loc[score].values[1]
#     tmp = num_review/2000
#     avy_num_review = num_review/num_home
#     size = num_home/tmp
    # Generate a uniform random sample using random
    data = list(set(sample2_rh_df[sample2_rh_df[aspect] == score].home_id))
    sample_home = np.random.choice(data, 75, replace=False)
    
    while not sample_validation(sample_home):
        sample_home = np.random.choice(data, 75, replace=False)
        if sample_validation(sample_home):
            break
        else:
            continue

    sample_home_review = sample2_rh_df[sample2_rh_df['home_id'].isin(sample_home)]
    print("Selected 75 homes among " + str(num_home) + " homes with score " + str(score) + ".\nSample home_ids:")
    print(sample_home)
    print("With in total: " + str(len(sample_home_review)) + ' reviews.')
    
    return sample_home_review

# This is a function is to guarantee:
# the number of reviews of the selected homes keeps around 1500.
def sample_validation(sample_home):
    sample_home_review = sample2_rh_df[sample2_rh_df['home_id'].isin(sample_home)]
    return True if np.abs(len(sample_home_review) - 2000) <= 100 else False

**********************************************************************************************
#### Get data for `cleanliness` aspect

Group homes with <= 7.0 cleanliness score together.
Based on literature review, the airbnb review score has positive bias. So, for score equal to or less than 7, we can assume there is a negetive implication of the homes.
As for homes with >= 9.0 score, randomly select sample as long as gurrantee 1500 number of reviews. 

In [32]:
clean_overview_df = overview_df('scores_cleanliness')
clean_overview_df
# clean_overview_df['number of reveiws']/clean_overview_df['number of homes']

,number of homes,number of reveiws
scores_cleanliness,,
3.0,1,2
4.0,4,9
5.0,5,23
6.0,30,107
7.0,40,511
8.0,183,3569
9.0,740,22197
10.0,2152,57264


In [33]:
# number of review selection around 1500.
clean_score7_df = sample2_rh_df[sample2_rh_df['scores_cleanliness'] <= 7.0]
print('Selected all homes with score no more than 7.0. \nSample home_ids:')
print(list(set(clean_score7_df.home_id)))
print("With in total: " + str(len(clean_score7_df)) + ' reviews.')
print('*' * 90)
clean_score8_df = home_sample_review(8.0, 'scores_cleanliness', clean_overview_df)
print('*' * 90)
clean_score9_df = home_sample_review(9.0, 'scores_cleanliness', clean_overview_df)
print('*' * 90)
clean_score10_df = home_sample_review(10.0, 'scores_cleanliness', clean_overview_df)
print('*' * 90)

clean_score7_df.to_csv('./Data/clean_score7_df.csv', sep='\t', encoding='utf-8')
clean_score8_df.to_csv('./Data/clean_score8_df.csv', sep='\t', encoding='utf-8')
clean_score9_df.to_csv('./Data/clean_score9_df.csv', sep='\t', encoding='utf-8')
clean_score10_df.to_csv('./Data/clean_score10_df.csv', sep='\t', encoding='utf-8')

Selected all homes with score no more than 7.0. 
Sample home_ids:
[7965184, 3291777, 6363779, 8754180, 3770248, 6370313, 1764233, 8594059, 4041868, 3766285, 9151374, 5479566, 666897, 7245586, 7934356, 4701205, 7093910, 1520533, 30712, 639130, 3889050, 3888924, 3534364, 716829, 6250399, 613151, 9509279, 5252515, 8934054, 4951079, 5353512, 3226793, 8083242, 8555304, 299817, 6865200, 7985714, 7203765, 5261239, 6992696, 670009, 8922554, 4340410, 8050232, 6864319, 2769088, 1520581, 670021, 3697351, 5639238, 4082250, 1520593, 8227710, 4550099, 7844444, 5126365, 9183838, 9519968, 8863714, 5078244, 6958436, 9075558, 6623079, 6959336, 7732071, 2856806, 5637990, 8067053, 4773614, 3593582, 6120046, 4892529, 3424242, 6717555, 7975026, 1145590, 3052151, 2357110, 3732094, 3096191]
With in total: 652 reviews.
******************************************************************************************
Selected 75 homes among 183 homes with score 8.0.
Sample home_ids:
[3732103 1815304 8391954 1950446 721

In [44]:
len(list(set(clean_score7_df.home_id)))

80

**********************************************************************************************
#### Get data for `location` aspect

Group homes with <= 7.0 cleanliness score together.

In [36]:
loc_overview_df = overview_df('scores_location')
loc_overview_df
# clean_overview_df['number of reveiws']/clean_overview_df['number of homes']

,number of homes,number of reveiws
scores_location,,
4.0,1,1
6.0,8,13
7.0,19,150
8.0,128,1833
9.0,884,29151
10.0,2115,52534


In [40]:
len(set(loc_score7_df.home_id))
# loc_overview_df['number of homes']

28

In [42]:
# number of review selection around 1500.
loc_score7_df = sample2_rh_df[sample2_rh_df['scores_location'] <= 7.0]
print('Selected all ' + str(len(set(loc_score7_df.home_id))) + ' homes with score no more than 7.0. \nSample home_ids:')
print(list(set(loc_score7_df.home_id)))
print("With in total: " + str(len(loc_score7_df)) + ' reviews.')
print('*' * 90)

loc_score8_df = sample2_rh_df[sample2_rh_df['scores_location'] == 8.0]
print('Selected all ' + str(len(set(loc_score8_df.home_id))) + ' homes with score no more than 7.0. \nSample home_ids:')
print(list(set(loc_score8_df.home_id)))
print("With in total: " + str(len(loc_score8_df)) + ' reviews.')
print('*' * 90)

loc_score9_df = home_sample_review(9.0, 'scores_location', loc_overview_df)
print('*' * 90)
loc_score10_df = home_sample_review(10.0, 'scores_location', loc_overview_df)
print('*' * 90)

loc_score7_df.to_csv('./Data/loc_score7_df.csv', sep='\t', encoding='utf-8')
loc_score8_df.to_csv('./Data/loc_score8_df.csv', sep='\t', encoding='utf-8')
loc_score9_df.to_csv('./Data/loc_score9_df.csv', sep='\t', encoding='utf-8')
loc_score10_df.to_csv('./Data/loc_score10_df.csv', sep='\t', encoding='utf-8')

Selected all 28 homes with score no more than 7.0. 
Sample home_ids:
[7965184, 8061699, 190984, 4041868, 1549973, 6215199, 613151, 6250399, 4951079, 5353512, 8083242, 5792683, 7071021, 6019762, 1022135, 23356, 4126284, 5104077, 6728017, 7415378, 6701018, 9183838, 3706719, 5637990, 6959336, 7839723, 1263470, 9157232]
With in total: 164 reviews.
******************************************************************************************
Selected all 128 homes with score no more than 7.0. 
Sample home_ids:
[8525825, 5219336, 2610187, 8988178, 2586642, 716829, 6325283, 6425652, 3726391, 3904056, 4395578, 7205443, 2986056, 7420488, 5407311, 7431247, 5340242, 2056276, 4589654, 7429207, 1246809, 606297, 9449062, 9134196, 264829, 4144767, 3291777, 4395654, 7048843, 877203, 3951768, 8934054, 611500, 278192, 823989, 4340410, 3849918, 5471427, 693956, 696004, 6249164, 6411986, 1461971, 7013085, 7500000, 3449059, 7388899, 6226666, 4708075, 9736940, 2586350, 571640, 1340668, 571651, 4163851, 666897, 